In [1]:
import pandas as pd
from transformers import AutoAdapterModel, AdapterConfig, AutoTokenizer
import torch
import numpy as np
from sklearn.metrics import f1_score

In [2]:
# en_train = pd.read_csv('translated_train_all.csv')
# en_test = pd.read_csv('translated_test_all.csv')

In [3]:
# en_train

In [4]:
am_train = pd.read_csv('am_train_translated.csv')
am_train, am_dev, am_test = np.split(
    am_train.sample(frac=1, random_state=42), [int(.7*len(am_train)), int(.8*len(am_train))])

am_train

,ID,tweet,label,eng_translated
1073,am_train_01074,ለትራምፕ ስንሳቀቅ ህውሀት ከመቀሌ ደሞ አልወሀድም ብላለች እና አሁን ህ...,negative,Habitheu says that when we laughed for the gru...
5468,am_train_05469,ለኔ ታላቅ ክብር ነው! እነበመባሌ የተሰማኝን ትልቅ ክብር በቃላት ልገልፀ...,positive,It is a great glory for me!I can't express it ...
5306,am_train_05307,በስህተት ወደ አካውንታቸው የገባውን አንድ ሚልዮን ብር በቅንነት የመለሱት...,positive,"In an error, the individual who first came to ..."
1321,am_train_01322,የሴቶች ደህንነት እንደ ዜጋ ቅድሚያ ይልተሰጠ... የችግኝን ያህል ግድ ያ...,negative,Women's Safety as a Citizen ... What is the pr...
4844,am_train_04845,ነገሮችን እንደአመጣጣቸው መቀበል የግድ ነው በዛው እሳቤ ኑሮ ትቂት ጊዜ...,positive,It is a necessity that we are coming as their ...
...,...,...,...,...
5700,am_train_05701,ጀግና ኢትዮጵያዊ ሁሌም ከእናንተ ጋር ነን,positive,Hero Ethiopian are always with you
5129,am_train_05130,ለመልካም ዕድልሲል በአውሮፕላን ሞተር ውስጥ ሳንቲም የወረወረው ግለሰብ ተ...,positive,The penny in a plane engine for fine lines!For...
3249,am_train_03250,ድርጊቱ መፈፀሙ ያልተገባ ቢሆንም ልዩ ልዩ ትንኮሳዎች ቢፈታተኑንም ቅሉ ዳ...,neutral,"Although the action is not unworthy, the struc..."
1208,am_train_01209,ትዕግስት ማለት ትዕግስት ማለት ብቻ አይደለም,negative,Trudy is not just Trudy


In [5]:
combined_train = pd.concat([
    am_train[['tweet', 'label']].rename(columns={'tweet':'text', 'label':'labels'})
#     en_train[['translated', 'labels']].rename(columns={'translated':'text'})
])

combined_test = pd.concat([
    am_test[['tweet', 'label']].rename(columns={'tweet':'text', 'label':'labels'}),
    am_dev[['tweet', 'label']].rename(columns={'tweet':'text', 'label':'labels'}),
#     en_test[['translated', 'labels']].rename(columns={'translated':'text'})
])
test_split_lengths = [('am_test', len(am_test)), ('am_dev', len(am_dev))]

In [21]:

label2id = {"positive":0, "neutral":1, 'negative':2}
id2label = {0:"positive", 1:"neutral", 2:'negative'}

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

def encode_batch(row):
    text = ' '.join(filter(lambda x:x[0]!='@', row.text.split()))
    out = tokenizer(text, max_length=100, truncation=True, padding="max_length", return_tensors='pt')
    out['labels'] = torch.LongTensor([label2id[row.labels]])[0]
    return out

train = combined_train.apply(encode_batch, axis=1).reset_index()[0]

len(train)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at /Users/thomas/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_versio

4188

In [22]:
test = combined_test.apply(encode_batch, axis=1).reset_index()[0]

len(test)

1796

In [23]:
model = AutoAdapterModel.from_pretrained('xlm-roberta-base')
model.add_adapter("sa")
model.train_adapter("sa")
model.add_classification_head("sa", num_labels=3)
model.set_active_adapters("sa")

loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at /Users/thomas/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading 

In [24]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True
#     # The next line is important to ensure the dataset labels are properly passed to the model
#     remove_unused_columns=False,
)

def compute_scores(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    i, output = 0, dict()
    for name, split_length in test_split_lengths:
        s = np.s_[i:i+split_length]
        split_preds = preds[s]
        split_labels = p.label_ids[s]
        output[f'{name}_acc'] = (split_preds==split_labels).mean()
        output[f'{name}_weighted_f1'] = f1_score(split_labels, split_preds, average='weighted')
        i += split_length
    return output

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_scores,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [25]:

trainer.train()

/Users/thomas/anaconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4188
  Num Epochs = 6
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 786


Step,Training Loss
200,0.998700
400,0.919900
600,0.888500


Saving model checkpoint to ./training_output/checkpoint-500
Configuration saved in ./training_output/checkpoint-500/sa/adapter_config.json
Module weights saved in ./training_output/checkpoint-500/sa/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-500/sa/head_config.json
Module weights saved in ./training_output/checkpoint-500/sa/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-500/sa/head_config.json
Module weights saved in ./training_output/checkpoint-500/sa/pytorch_model_head.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=786, training_loss=0.9199164587122793, metrics={'train_runtime': 12861.8233, 'train_samples_per_second': 1.954, 'train_steps_per_second': 0.061, 'total_flos': 1313702177659200.0, 'train_loss': 0.9199164587122793, 'epoch': 6.0})

In [26]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1796
  Batch size = 32


{'eval_loss': 0.8505920767784119,
 'eval_am_test_acc': 0.6274018379281537,
 'eval_am_test_weighted_f1': 0.601068064734614,
 'eval_am_dev_acc': 0.5826377295492488,
 'eval_am_dev_weighted_f1': 0.5542698085979245,
 'eval_runtime': 424.5438,
 'eval_samples_per_second': 4.23,
 'eval_steps_per_second': 0.134,
 'epoch': 6.0}